# W&B Results Fetcher

Fetches pass@k metrics directly from W&B for final experiments.

In [1]:
import wandb
import pandas as pd
from IPython.display import display, Markdown

# W&B settings
ENTITY = "bdsaglam"

# Projects to fetch from (add all relevant projects)
PROJECTS = {
    "semi-final": "etrm-semi-final-subset-eval",
    "final": "etrm-final",
    "variational-kl-fix": "variational-kl-fix",
}

# Metrics to extract
PASS_K_METRICS = ["ARC/pass@1", "ARC/pass@2", "ARC/pass@5", "ARC/pass@10", "ARC/pass@100", "ARC/pass@1000"]

In [2]:
def fetch_runs(entity: str, project: str, filters: dict = None) -> list:
    """Fetch runs from W&B project."""
    api = wandb.Api()
    runs = api.runs(f"{entity}/{project}", filters=filters)
    return list(runs)


def extract_pass_metrics(run) -> dict:
    """Extract pass@k metrics from a run's summary."""
    summary = run.summary
    
    result = {
        "run_id": run.id,
        "run_name": run.name,
        "state": run.state,
        "created_at": run.created_at,
    }
    
    # Extract pass@k metrics
    for metric in PASS_K_METRICS:
        result[metric] = summary.get(metric)
    
    # Extract training metrics
    result["train/exact_accuracy"] = summary.get("train/exact_accuracy")
    result["_step"] = summary.get("_step")
    result["_runtime"] = summary.get("_runtime")
    
    # Extract config info
    config = run.config
    arch = config.get("arch", {})
    result["encoder_type"] = arch.get("encoder_type", arch.get("recurrent_encoder_type", "unknown"))
    result["encoder_num_layers"] = arch.get("encoder_num_layers")
    result["epochs"] = config.get("epochs")
    result["global_batch_size"] = config.get("global_batch_size")
    result["max_eval_groups"] = config.get("max_eval_groups")
    result["max_train_groups"] = config.get("max_train_groups")
    result["kl_weight"] = arch.get("loss", {}).get("kl_weight")
    
    return result


def runs_to_dataframe(runs: list) -> pd.DataFrame:
    """Convert list of runs to DataFrame."""
    records = [extract_pass_metrics(run) for run in runs]
    df = pd.DataFrame(records)
    return df

## Fetch All Runs

In [3]:
# Fetch runs from all projects
all_runs = {}
all_dfs = {}

for project_key, project_name in PROJECTS.items():
    print(f"Fetching runs from {project_name}...")
    try:
        runs = fetch_runs(ENTITY, project_name)
        all_runs[project_key] = runs
        all_dfs[project_key] = runs_to_dataframe(runs)
        print(f"  Found {len(runs)} runs")
    except Exception as e:
        print(f"  Error: {e}")
        all_runs[project_key] = []
        all_dfs[project_key] = pd.DataFrame()

Fetching runs from etrm-semi-final-subset-eval...


wandb: Currently logged in as: bdsaglam to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


  Found 17 runs
Fetching runs from etrm-final...
  Found 4 runs
Fetching runs from variational-kl-fix...
  Found 6 runs


## Semi-Final Results

In [4]:
if "semi-final" in all_dfs and not all_dfs["semi-final"].empty:
    df = all_dfs["semi-final"].copy()
    
    # Filter to finished runs with pass@1 metrics
    df = df[df["ARC/pass@1"].notna()]
    
    # Sort by pass@1 descending
    df = df.sort_values("ARC/pass@1", ascending=False)
    
    # Display
    display_cols = ["run_name", "encoder_type", "ARC/pass@1", "ARC/pass@2", "ARC/pass@5", 
                   "train/exact_accuracy", "_step", "state"]
    display_cols = [c for c in display_cols if c in df.columns]
    
    print(f"Semi-final runs with pass@k metrics: {len(df)}")
    display(df[display_cols])
else:
    print("No semi-final runs found")

Semi-final runs with pass@k metrics: 15


,run_name,encoder_type,ARC/pass@1,ARC/pass@2,ARC/pass@5,train/exact_accuracy,_step,state
8,SF5_standard_baseline,standard,0.0100,0.0100,0.0150,0.433594,3492,finished
14,LPN2_paper_var,lpn_var,0.0075,0.0075,0.0075,0.121094,1746,finished
7,SF1_hybrid_std_baseline,hybrid_standard,0.0050,0.0075,0.0100,0.375000,6970,finished
2,SF1_hybrid_std_baseline,hybrid_standard,0.0000,0.0000,0.0000,0.031250,6967,finished
3,SF2_hybrid_std_explore0.7,hybrid_standard,0.0000,0.0000,0.0000,0.023438,6967,finished
1,SF6_standard_explore0.7,standard,0.0000,0.0000,0.0000,0.042969,3490,finished
0,SF5_standard_baseline,standard,0.0000,0.0000,0.0000,0.027344,3490,finished
5,SF4_hybrid_var_explore0.7,hybrid_variational,0.0000,0.0000,0.0000,0.000000,6967,finished
4,SF3_hybrid_var_baseline,hybrid_variational,0.0000,0.0000,0.0000,0.000000,6967,finished
10,SF3_hybrid_var,hybrid_variational,0.0000,0.0000,0.0000,0.007812,6970,finished


## Final Results

In [5]:
if "final" in all_dfs and not all_dfs["final"].empty:
    df = all_dfs["final"].copy()
    
    # Filter to runs with pass@1 metrics
    df = df[df["ARC/pass@1"].notna()]
    
    # Sort by pass@1 descending
    df = df.sort_values("ARC/pass@1", ascending=False)
    
    # Display
    display_cols = ["run_name", "encoder_type", "ARC/pass@1", "ARC/pass@2", "ARC/pass@5", 
                   "train/exact_accuracy", "_step", "epochs", "state"]
    display_cols = [c for c in display_cols if c in df.columns]
    
    print(f"Final runs with pass@k metrics: {len(df)}")
    display(df[display_cols])
else:
    print("No final runs found")

Final runs with pass@k metrics: 3


,run_name,encoder_type,ARC/pass@1,ARC/pass@2,ARC/pass@5,train/exact_accuracy,_step,epochs,state
0,F1_standard,standard,0.0,0.0050,0.0050,0.789062,174622,50000,finished
1,F2_hybrid_var,hybrid_variational,0.0,0.0000,0.0000,0.406250,174240,25000,finished
2,F3_etrmtrm,standard,0.0,0.0025,0.0025,0.511719,87310,25000,finished


## Generate Markdown Tables

In [6]:
def format_pct(val):
    """Format value as percentage."""
    if pd.isna(val):
        return "-"
    return f"{val * 100:.2f}%"


def generate_results_table(df: pd.DataFrame, title: str) -> str:
    """Generate markdown table from results DataFrame."""
    if df.empty:
        return f"## {title}\n\nNo results available."
    
    # Filter to runs with pass@1
    df = df[df["ARC/pass@1"].notna()].copy()
    if df.empty:
        return f"## {title}\n\nNo runs with pass@k metrics."
    
    # Sort by pass@1
    df = df.sort_values("ARC/pass@1", ascending=False)
    
    lines = [f"## {title}", "", "| Run | Encoder | Pass@1 | Pass@2 | Pass@5 | Steps |",
             "|-----|---------|--------|--------|--------|-------|"]    
    
    for _, row in df.iterrows():
        run_name = row.get("run_name", "unknown")
        encoder = row.get("encoder_type", "unknown")
        pass1 = format_pct(row.get("ARC/pass@1"))
        pass2 = format_pct(row.get("ARC/pass@2"))
        pass5 = format_pct(row.get("ARC/pass@5"))
        steps = row.get("_step", "-")
        if pd.notna(steps):
            steps = f"{int(steps):,}"
        
        lines.append(f"| {run_name} | {encoder} | {pass1} | {pass2} | {pass5} | {steps} |")
    
    return "\n".join(lines)

In [7]:
# Generate tables
for project_key, df in all_dfs.items():
    table = generate_results_table(df, f"{project_key.title()} Results")
    display(Markdown(table))
    print("\n" + "="*60 + "\n")

## Semi-Final Results

| Run | Encoder | Pass@1 | Pass@2 | Pass@5 | Steps |
|-----|---------|--------|--------|--------|-------|
| SF5_standard_baseline | standard | 1.00% | 1.00% | 1.50% | 3,492 |
| LPN2_paper_var | lpn_var | 0.75% | 0.75% | 0.75% | 1,746 |
| SF1_hybrid_std_baseline | hybrid_standard | 0.50% | 0.75% | 1.00% | 6,970 |
| SF1_hybrid_std_baseline | hybrid_standard | 0.00% | 0.00% | 0.00% | 6,967 |
| SF2_hybrid_std_explore0.7 | hybrid_standard | 0.00% | 0.00% | 0.00% | 6,967 |
| SF6_standard_explore0.7 | standard | 0.00% | 0.00% | 0.00% | 3,490 |
| SF5_standard_baseline | standard | 0.00% | 0.00% | 0.00% | 3,490 |
| SF4_hybrid_var_explore0.7 | hybrid_variational | 0.00% | 0.00% | 0.00% | 6,967 |
| SF3_hybrid_var_baseline | hybrid_variational | 0.00% | 0.00% | 0.00% | 6,967 |
| SF3_hybrid_var | hybrid_variational | 0.00% | 0.00% | 0.00% | 6,970 |
| SF3_hybrid_var_baseline | hybrid_variational | 0.00% | 0.00% | 0.00% | 6,970 |
| SF3_hybrid_var_kl_3 | hybrid_variational | 0.00% | 0.00% | 0.00% | 6,970 |
| SF3_hybrid_var_kl_2 | hybrid_variational | 0.00% | 0.00% | 0.00% | 6,970 |
| LPN3_deep_std | lpn_standard | 0.00% | 0.00% | 0.00% | 6,941 |
| LPN4_deep_var | lpn_variational | 0.00% | 0.00% | 0.00% | 6,941 |

## Final Results

| Run | Encoder | Pass@1 | Pass@2 | Pass@5 | Steps |
|-----|---------|--------|--------|--------|-------|
| F1_standard | standard | 0.00% | 0.50% | 0.50% | 174,622 |
| F2_hybrid_var | hybrid_variational | 0.00% | 0.00% | 0.00% | 174,240 |
| F3_etrmtrm | standard | 0.00% | 0.25% | 0.25% | 87,310 |

## Variational-Kl-Fix Results

| Run | Encoder | Pass@1 | Pass@2 | Pass@5 | Steps |
|-----|---------|--------|--------|--------|-------|
| VKL1_lpn_var_kl0.0001_lpn_paper | lpn_variational | 0.00% | 0.00% | 0.00% | 5,000 |
| VKL2_lpn_var_kl0.001 | lpn_variational | 0.00% | 0.00% | 0.00% | 5,000 |
| VKL5_hybrid_var_kl0.0001_lpn_paper | hybrid_variational | 0.00% | 0.00% | 0.00% | 5,000 |
| VKL0_standard_control | standard | 0.00% | 0.00% | 0.00% | 2,500 |
| VKL6_lpn_var_kl1e-5 | lpn_variational | 0.00% | 0.00% | 0.00% | 5,000 |

## Detailed Run Info

In [8]:
def get_run_details(run) -> dict:
    """Get detailed info from a run."""
    summary = run.summary
    config = run.config
    
    return {
        "run_id": run.id,
        "run_name": run.name,
        "state": run.state,
        "url": run.url,
        # Pass@k
        "ARC/pass@1": summary.get("ARC/pass@1"),
        "ARC/pass@2": summary.get("ARC/pass@2"),
        "ARC/pass@5": summary.get("ARC/pass@5"),
        "ARC/pass@10": summary.get("ARC/pass@10"),
        "ARC/pass@100": summary.get("ARC/pass@100"),
        "ARC/pass@1000": summary.get("ARC/pass@1000"),
        # Training
        "train/exact_accuracy": summary.get("train/exact_accuracy"),
        "train/accuracy": summary.get("train/accuracy"),
        "_step": summary.get("_step"),
        "_runtime": summary.get("_runtime"),
        # Config
        "encoder_type": config.get("arch", {}).get("encoder_type"),
        "encoder_num_layers": config.get("arch", {}).get("encoder_num_layers"),
        "epochs": config.get("epochs"),
        "global_batch_size": config.get("global_batch_size"),
        "lr": config.get("lr"),
        "max_eval_groups": config.get("max_eval_groups"),
        "max_train_groups": config.get("max_train_groups"),
        "ema": config.get("ema"),
    }

In [9]:
# Get detailed info for final runs
if "final" in all_runs and all_runs["final"]:
    details = [get_run_details(run) for run in all_runs["final"]]
    details_df = pd.DataFrame(details)
    
    # Filter to runs with pass@1
    details_df = details_df[details_df["ARC/pass@1"].notna()]
    details_df = details_df.sort_values("ARC/pass@1", ascending=False)
    
    print("Detailed Final Run Info:")
    display(details_df)

Detailed Final Run Info:


,run_id,run_name,state,url,ARC/pass@1,ARC/pass@2,ARC/pass@5,ARC/pass@10,ARC/pass@100,ARC/pass@1000,...,_step,_runtime,encoder_type,encoder_num_layers,epochs,global_batch_size,lr,max_eval_groups,max_train_groups,ema
0,z31hae14,F1_standard,finished,https://wandb.ai/bdsaglam/etrm-final/runs/z31h...,0.0,0.0050,0.0050,0.0050,0.0050,0.0075,...,174622,95286.127706,standard,2,50000,256,0.0001,32,None,True
1,7km7llbl,F2_hybrid_var,finished,https://wandb.ai/bdsaglam/etrm-final/runs/7km7...,0.0,0.0000,0.0000,0.0000,0.0000,0.0000,...,174240,82114.666997,hybrid_variational,4,25000,128,0.0001,32,None,True
2,wj3xu8md,F3_etrmtrm,finished,https://wandb.ai/bdsaglam/etrm-final/runs/wj3x...,0.0,0.0025,0.0025,0.0025,0.0025,0.0025,...,87310,26121.732932,standard,2,25000,256,0.0001,32,None,True


## Export Results

In [10]:
# Export to CSV
import os

output_dir = "./outputs"
os.makedirs(output_dir, exist_ok=True)

for project_key, df in all_dfs.items():
    if not df.empty:
        output_path = os.path.join(output_dir, f"wandb_results_{project_key}.csv")
        df.to_csv(output_path, index=False)
        print(f"Saved {project_key} results to {output_path}")

Saved semi-final results to ./outputs/wandb_results_semi-final.csv
Saved final results to ./outputs/wandb_results_final.csv
Saved variational-kl-fix results to ./outputs/wandb_results_variational-kl-fix.csv


## Summary Table for Report

In [11]:
def compute_data_passes(steps, batch_size, num_train_groups, mean_examples_per_group=912):
    """Compute approximate number of data passes (epochs) from steps.
    
    Args:
        steps: Number of training steps
        batch_size: Global batch size
        num_train_groups: Number of training puzzle groups (e.g., 560 for full dataset)
        mean_examples_per_group: Mean augmented examples per group (~912 for aug-1000)
    
    Returns:
        Approximate number of full passes through training data
    """
    if pd.isna(steps) or pd.isna(batch_size):
        return None
    total_examples = num_train_groups * mean_examples_per_group
    examples_seen = steps * batch_size
    return examples_seen / total_examples


def create_report_table(all_dfs: dict, num_train_groups: int = 560) -> pd.DataFrame:
    """Create a combined summary table for the report."""
    rows = []
    
    for project_key, df in all_dfs.items():
        if df.empty:
            continue
        
        # Filter to runs with pass@1
        df = df[df["ARC/pass@1"].notna()].copy()
        
        for _, row in df.iterrows():
            batch_size = row.get("global_batch_size", 256)
            steps = row.get("_step")
            data_passes = compute_data_passes(steps, batch_size, num_train_groups)
            
            rows.append({
                "Project": project_key,
                "Run": row.get("run_name", "unknown"),
                "Encoder": row.get("encoder_type", "unknown"),
                "Pass@1": row.get("ARC/pass@1"),
                "Pass@2": row.get("ARC/pass@2"),
                "Pass@5": row.get("ARC/pass@5"),
                "Train Acc": row.get("train/exact_accuracy"),
                "Steps": steps,
                "Data Passes": data_passes,
                "State": row.get("state", "unknown"),
            })
    
    if not rows:
        return pd.DataFrame()
    
    summary_df = pd.DataFrame(rows)
    summary_df = summary_df.sort_values(["Project", "Pass@1"], ascending=[True, False])
    
    return summary_df


# Create report table (raw values)
report_table_raw = create_report_table(all_dfs)
if not report_table_raw.empty:
    print(f"Found {len(report_table_raw)} runs with pass@k metrics")
    display(report_table_raw)
else:
    print("No results available")

Found 23 runs with pass@k metrics


,Project,Run,Encoder,Pass@1,Pass@2,Pass@5,Train Acc,Steps,Data Passes,State
15,final,F1_standard,standard,0.0000,0.0050,0.0050,0.789062,174622,87.529825,finished
16,final,F2_hybrid_var,hybrid_variational,0.0000,0.0000,0.0000,0.406250,174240,43.669173,finished
17,final,F3_etrmtrm,standard,0.0000,0.0025,0.0025,0.511719,87310,43.764411,finished
7,semi-final,SF5_standard_baseline,standard,0.0100,0.0100,0.0150,0.433594,3492,1.750376,finished
12,semi-final,LPN2_paper_var,lpn_var,0.0075,0.0075,0.0075,0.121094,1746,0.875188,finished
6,semi-final,SF1_hybrid_std_baseline,hybrid_standard,0.0050,0.0075,0.0100,0.375000,6970,1.746867,finished
0,semi-final,SF5_standard_baseline,standard,0.0000,0.0000,0.0000,0.027344,3490,1.749373,finished
1,semi-final,SF6_standard_explore0.7,standard,0.0000,0.0000,0.0000,0.042969,3490,1.749373,finished
2,semi-final,SF1_hybrid_std_baseline,hybrid_standard,0.0000,0.0000,0.0000,0.031250,6967,1.746115,finished
3,semi-final,SF2_hybrid_std_explore0.7,hybrid_standard,0.0000,0.0000,0.0000,0.023438,6967,1.746115,finished


In [12]:
# Format table for display/export
def format_report_table(df: pd.DataFrame) -> pd.DataFrame:
    """Format report table with percentages and number formatting."""
    if df.empty:
        return df
    
    df = df.copy()
    
    # Format percentages
    for col in ["Pass@1", "Pass@2", "Pass@5", "Train Acc"]:
        if col in df.columns:
            df[col] = df[col].apply(lambda x: f"{x*100:.2f}%" if pd.notna(x) else "-")
    
    # Format steps
    if "Steps" in df.columns:
        df["Steps"] = df["Steps"].apply(lambda x: f"{int(x):,}" if pd.notna(x) else "-")
    
    # Format data passes
    if "Data Passes" in df.columns:
        df["Data Passes"] = df["Data Passes"].apply(lambda x: f"{x:,.0f}" if pd.notna(x) else "-")
    
    return df


if not report_table_raw.empty:
    # Format for display
    report_table_formatted = format_report_table(report_table_raw)
    
    # Generate markdown
    print("Markdown table for report:\n")
    print(report_table_formatted.to_markdown(index=False))
    
    # Also display as DataFrame
    print("\n\nFormatted table:")
    display(report_table_formatted)
else:
    print("No results to format")

Markdown table for report:

| Project            | Run                                | Encoder            | Pass@1   | Pass@2   | Pass@5   | Train Acc   | Steps   |   Data Passes | State    |
|:-------------------|:-----------------------------------|:-------------------|:---------|:---------|:---------|:------------|:--------|--------------:|:---------|
| final              | F1_standard                        | standard           | 0.00%    | 0.50%    | 0.50%    | 78.91%      | 174,622 |            88 | finished |
| final              | F2_hybrid_var                      | hybrid_variational | 0.00%    | 0.00%    | 0.00%    | 40.62%      | 174,240 |            44 | finished |
| final              | F3_etrmtrm                         | standard           | 0.00%    | 0.25%    | 0.25%    | 51.17%      | 87,310  |            44 | finished |
| semi-final         | SF5_standard_baseline              | standard           | 1.00%    | 1.00%    | 1.50%    | 43.36%      | 3,492   |          

,Project,Run,Encoder,Pass@1,Pass@2,Pass@5,Train Acc,Steps,Data Passes,State
15,final,F1_standard,standard,0.00%,0.50%,0.50%,78.91%,"174,622",88,finished
16,final,F2_hybrid_var,hybrid_variational,0.00%,0.00%,0.00%,40.62%,"174,240",44,finished
17,final,F3_etrmtrm,standard,0.00%,0.25%,0.25%,51.17%,"87,310",44,finished
7,semi-final,SF5_standard_baseline,standard,1.00%,1.00%,1.50%,43.36%,"3,492",2,finished
12,semi-final,LPN2_paper_var,lpn_var,0.75%,0.75%,0.75%,12.11%,"1,746",1,finished
6,semi-final,SF1_hybrid_std_baseline,hybrid_standard,0.50%,0.75%,1.00%,37.50%,"6,970",2,finished
0,semi-final,SF5_standard_baseline,standard,0.00%,0.00%,0.00%,2.73%,"3,490",2,finished
1,semi-final,SF6_standard_explore0.7,standard,0.00%,0.00%,0.00%,4.30%,"3,490",2,finished
2,semi-final,SF1_hybrid_std_baseline,hybrid_standard,0.00%,0.00%,0.00%,3.12%,"6,967",2,finished
3,semi-final,SF2_hybrid_std_explore0.7,hybrid_standard,0.00%,0.00%,0.00%,2.34%,"6,967",2,finished


## Final Experiments Table (F1-F4)

In [13]:
# Filter for final experiments (F1-F4)
FINAL_RUNS = ["F1_standard", "F2_hybrid_var", "F3_etrmtrm", "F4_lpn_var"]

if "final" in all_dfs and not all_dfs["final"].empty:
    df = all_dfs["final"].copy()
    
    # Filter to F1-F4 runs
    df_final = df[df["run_name"].isin(FINAL_RUNS)].copy()
    
    if not df_final.empty:
        # Compute data passes
        df_final["data_passes"] = df_final.apply(
            lambda row: compute_data_passes(
                row["_step"], 
                row.get("global_batch_size", 256), 
                560  # Full training set groups
            ), 
            axis=1
        )
        
        # Create clean table
        final_table = df_final[[
            "run_name", "encoder_type", 
            "ARC/pass@1", "ARC/pass@2", "ARC/pass@5",
            "train/exact_accuracy", "_step", "data_passes", "state"
        ]].copy()
        
        final_table.columns = [
            "Run", "Encoder", 
            "Pass@1", "Pass@2", "Pass@5",
            "Train Acc", "Steps", "Data Passes", "State"
        ]
        
        # Sort by run name (F1, F2, F3, F4)
        final_table["sort_key"] = final_table["Run"].apply(lambda x: FINAL_RUNS.index(x) if x in FINAL_RUNS else 99)
        final_table = final_table.sort_values("sort_key").drop("sort_key", axis=1)
        
        print("Final Experiments (F1-F4):")
        print("-" * 60)
        display(final_table)
        
        # Format for report
        formatted = final_table.copy()
        for col in ["Pass@1", "Pass@2", "Pass@5", "Train Acc"]:
            if col in formatted.columns:
                formatted[col] = formatted[col].apply(lambda x: f"{x*100:.2f}%" if pd.notna(x) else "-")
        formatted["Steps"] = formatted["Steps"].apply(lambda x: f"{int(x):,}" if pd.notna(x) else "-")
        formatted["Data Passes"] = formatted["Data Passes"].apply(lambda x: f"{x:,.0f}" if pd.notna(x) else "-")
        
        print("\n\nFormatted for report:")
        print(formatted.to_markdown(index=False))
    else:
        print("No F1-F4 runs found in final project")
else:
    print("Final project not loaded")

Final Experiments (F1-F4):
------------------------------------------------------------


,Run,Encoder,Pass@1,Pass@2,Pass@5,Train Acc,Steps,Data Passes,State
0,F1_standard,standard,0.0,0.0050,0.0050,0.789062,174622,87.529825,finished
1,F2_hybrid_var,hybrid_variational,0.0,0.0000,0.0000,0.406250,174240,43.669173,finished
2,F3_etrmtrm,standard,0.0,0.0025,0.0025,0.511719,87310,43.764411,finished
3,F4_lpn_var,lpn_var,NaN,NaN,NaN,0.363281,6379,3.197494,running




Formatted for report:
| Run           | Encoder            | Pass@1   | Pass@2   | Pass@5   | Train Acc   | Steps   |   Data Passes | State    |
|:--------------|:-------------------|:---------|:---------|:---------|:------------|:--------|--------------:|:---------|
| F1_standard   | standard           | 0.00%    | 0.50%    | 0.50%    | 78.91%      | 174,622 |            88 | finished |
| F2_hybrid_var | hybrid_variational | 0.00%    | 0.00%    | 0.00%    | 40.62%      | 174,240 |            44 | finished |
| F3_etrmtrm    | standard           | 0.00%    | 0.25%    | 0.25%    | 51.17%      | 87,310  |            44 | finished |
| F4_lpn_var    | lpn_var            | -        | -        | -        | 36.33%      | 6,379   |             3 | running  |
